In [1]:
# Required Libraries
import boto3
import botocore

import tarfile

import time

import os
from os import path
import shutil
import glob

In [2]:
#### Required functions

def delete(file='',folder=''):
    
    if folder:
        if path.exists(folder):
            shutil.rmtree(folder)
            print('------Deleted the folder: ',folder,' ------')
        else:
            return
    
    elif file:
        if path.exists(file):
            os.remove(file)
            print('------Deleted the file: ',file,' ------')
        else:
            return
    
    else:
        print('Invalid Path')

def download_file_from_s3(bucket_name='',file_name='',new_file_name=''):
    s3 = boto3.resource('s3')
    
    try:
        delete(file=new_file_name)
        s3.Bucket(bucket_name).download_file(file_name,new_file_name)
        print('---Downloaded Sucessfully---')
    
    except botocore.exceptions.ClientErrors as e:
        
        if e.response['Error']['Code'] == '404':
            print('The object does not exist')
        else:
            print('---Error in Downloading the S3 file---')

def extracting_file(file_name=''):
    
    if file_name:
        
        delete(folder=folder_name_local)
        
        my_tar = tarfile.open(file_name)
        my_tar.extractall()
        my_tar.close()
        delete(file=file_name_local)
        
        print('------ Extracted Successfully-----')
    else:
        print('Invalid address / file_name')

## Downloading and Extracting the CJP Data

In [32]:
start_time = time.time()

In [33]:
file_name_s3 = 'cjp-tables-af3a7aefbc195b871992.tar.gz'
file_name_local = 'cjp_data_file.tar.gz'
folder_name_local = 'cjp_tables'
s3_bucket_name = 'cjp-news-data'

In [34]:
%%time
downloading_time_start = time.time()
download_file_from_s3(bucket_name=s3_bucket_name,
                     file_name=file_name_s3,
                     new_file_name=file_name_local)
downloading_time_end = time.time()

------Deleted the file:  cjp_data_file.tar.gz  ------
---Downloaded Sucessfully---
Wall time: 2min 37s


In [35]:
%%time
download_and_extracting_time_start = time.time()
extracting_file(file_name=file_name_local)
download_and_extracting_time_end = time.time()

------Deleted the folder:  cjp_tables  ------
------Deleted the file:  cjp_data_file.tar.gz  ------
------ Extracted Successfully-----
Wall time: 51.5 s


## Extracting the required CJP NEWS Data From Downladed S3 file (2017-Present)

In [2]:
#%cd /Users/mchladek/Dev/article-tagging/lib/tagnews/

from req_file import load_all_data, download_data
import pandas as pd
import numpy as np
import datetime

SET GLOBAL VARIABLES

In [3]:
CATEGORY_RELEVANCE_MIN = 0.50

LOAD DATA

In [4]:
#%cd /Users/mchladek/Dev/chicago-justice-data/
data_loading_time_start = time.time()
newssource_df, articles_df, categories_df, trainedcategoryrelevance_df, trainedcoding_df, usercoding_df,  usercoding_categories_df, trainedlocation_df, trainedsentiment_df, trainedsentimententities_df = load_all_data()
data_loading_time_end = time.time()


news sources loaded. size: (39, 4)


C:\Users\sukes\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3418: DtypeWarning: Columns (1,3,10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


articles loaded. size: (880325, 11)
categories loaded. size: (40, 6)
trainedcategoryrelevance loaded. size: (1909987, 4)
trainedlocation loaded. size: (496122, 8)
trainedcoding loaded. size: (880325, 8)
usercoding loaded. size: (293663, 7)
usercoding_categories loaded. size: (158622, 3)
trainedsentiment loaded. size: (12660, 4)
trainedsentimententities loaded. size: (69610, 6)


### Looking into the Data - Practicum student code

In [5]:
newssource_df.head()

,source_id,source_name,short_name,legacy_feed_id
0,97,Unknown,unknown,NaN
1,98,ABC Local,abc-local,A
2,99,Beachwood Reporter,beachwood-reporter,C
3,100,CBS Local,cbs-local,L
4,101,Chicago Defender,chicago-defender,D


In [6]:
articles_df.head()

,id,feedname,url,orig_html,title,bodytext,relevant,created,last_modified,news_source_id,author
0,457913,NaN,http://www.chicagotribune.com/ct-ahp-arlington...,"<div class=""trb_ar_bd"" itemprop=""articleBody"">...",65-year-old homeless man found dead in public ...,No foul play is suspected in the death of a 65...,t,2018-05-11 02:01:08.696534+00,2018-05-11 02:01:08.696551+00,113,Karen Ann Cullotta
1,457914,NaN,http://www.chicagotribune.com/ct-spt-bears-roo...,"<div class=""trb_ar_bd"" itemprop=""articleBody"">...","Bears sign 6 draft picks, announce rookie mini...",The [ Bears ][12] have signed six of their sev...,t,2018-05-11 02:01:09.868967+00,2018-05-11 02:01:09.868985+00,113,Colleen Kane
2,457915,NaN,http://www.chicagotribune.com/ct-edit-lincoln-...,"<div class=""trb_ar_bd"" itemprop=""articleBody"">...","Lincoln Towing: 8 months, 831 violations.",Lincoln Towing’s reputation as a predatory sco...,t,2018-05-11 02:01:11.023647+00,2018-05-11 02:01:11.023663+00,113,Editorial Board
3,457916,NaN,http://www.chicagotribune.com/ct-met-abraham-l...,"<div class=""trb_ar_bd"" itemprop=""articleBody"">...",Historical Lincoln items could go on auction b...,The Land of Lincoln is in such financial disar...,t,2018-05-11 02:01:12.36763+00,2018-05-11 02:01:12.367646+00,113,Ray Long
4,457917,NaN,http://www.chicagotribune.com/ct-sta-joshua-hu...,"<div class=""trb_ar_bd"" itemprop=""articleBody"">...",Ex-Saint Xavier University football player sho...,Joshua Hunter had an infectious personality an...,t,2018-05-11 02:01:13.629099+00,2018-05-11 02:01:13.629117+00,113,Steve Millar


In [7]:
articles_df.tail()

,id,feedname,url,orig_html,title,bodytext,relevant,created,last_modified,news_source_id,author
880320,457909,NaN,https://chicago.suntimes.com/news/man-boy-woun...,"<div class=""post-content columns small-12 larg...","Man, boy wounded in Uptown shooting | Chicago ...",A man and a boy were shot Thursday evening in ...,t,2018-05-11 02:01:03.157893+00,2018-05-11 02:01:03.157914+00,112,Nader Issa
880321,457910,NaN,https://chicago.suntimes.com/news/pedestrian-k...,"<div class=""post-content columns small-12 larg...","Pedestrian killed, another injured by car flee...",A pedestrian was killed and another was injure...,t,2018-05-11 02:01:04.876685+00,2018-05-11 02:01:04.876703+00,112,Nader Issa
880322,457911,NaN,http://www.chicagotribune.com/ct-met-three-hur...,"<div class=""trb_ar_bd"" itemprop=""articleBody"">...","Police: Woman, 55, dead after car fleeing traf...",A woman has died after a car that was pulled o...,t,2018-05-11 02:01:06.108941+00,2018-05-11 02:01:06.10896+00,113,Deanese Williams-Harris
880323,457912,NaN,http://www.chicagotribune.com/ct-met-tom-ricke...,"<div class=""trb_ar_bd"" itemprop=""articleBody"">...",Chicago Fire fans want Tom Ricketts to buy the...,So what do [ Chicago Fire ][1] fans think of [...,t,2018-05-11 02:01:07.40272+00,2018-05-11 02:01:07.402738+00,113,John Kass
880324,457952,NaN,http://www.fox32chicago.com/news/local/two-chi...,"<div class=""mod-content story-content"" data-in...","Two Chicago officers, Good Samaritan honored f...",![photo][9]\n\n![photo][9] _ _\n\nTwo Chicago ...,t,2018-05-11 04:01:25.776076+00,2018-05-11 04:01:25.776092+00,109,NaN


In [8]:
articles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 880325 entries, 0 to 880324
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   id              880325 non-null  int64 
 1   feedname        292158 non-null  object
 2   url             880325 non-null  object
 3   orig_html       586530 non-null  object
 4   title           880299 non-null  object
 5   bodytext        874303 non-null  object
 6   relevant        880325 non-null  object
 7   created         880325 non-null  object
 8   last_modified   880325 non-null  object
 9   news_source_id  880325 non-null  int64 
 10  author          391765 non-null  object
dtypes: int64(2), object(9)
memory usage: 73.9+ MB


#### Coverting req columns to date type

In [9]:
# Converting last_modified and created columns to date-type
articles_df['last_modified'] = pd.to_datetime(articles_df['last_modified']).dt.tz_convert(None)
articles_df['created'] = pd.to_datetime(articles_df['created']).dt.tz_convert(None)

In [10]:
articles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 880325 entries, 0 to 880324
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   id              880325 non-null  int64         
 1   feedname        292158 non-null  object        
 2   url             880325 non-null  object        
 3   orig_html       586530 non-null  object        
 4   title           880299 non-null  object        
 5   bodytext        874303 non-null  object        
 6   relevant        880325 non-null  object        
 7   created         880325 non-null  datetime64[ns]
 8   last_modified   880325 non-null  datetime64[ns]
 9   news_source_id  880325 non-null  int64         
 10  author          391765 non-null  object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 73.9+ MB


### Sorting the data-frame based on date - Descinding Order

In [11]:
articles_df.sort_values(by="created",ascending=False,inplace=True)

In [12]:
articles_df.head()

,id,feedname,url,orig_html,title,bodytext,relevant,created,last_modified,news_source_id,author
422577,880836,NaN,http://www.chicagobusiness.com/manufacturing/b...,"<div class=""field--name-field-paragraph-body ""...",Boeing finds electrical issue in new areas of ...,(Bloomberg) — Boeing Co. has found potential e...,t,2021-04-17 06:00:42.398188,2021-04-17 06:00:42.398201,107,NaN
422576,880835,NaN,https://chicago.suntimes.com/news/2021/4/16/22...,"<div class=""c-entry-content piano-paywall-hide...",16-year-old girl fatally struck by vehicle in ...,A 16-year-old girl was fatally hit my a vehicl...,t,2021-04-17 06:00:37.066859,2021-04-17 06:00:37.066875,112,Sun-Times Wire
422575,880834,NaN,https://chicago.suntimes.com/2021/4/17/2238276...,"<div class=""c-entry-content piano-paywall-hide...","Horoscope for Saturday, April 17, 2021",## Moon Alert\n\nAvoid shopping or making imp...,t,2021-04-17 06:00:35.583052,2021-04-17 06:00:35.583068,112,Georgia Nicols
422574,880833,NaN,https://chicago.cbslocal.com/2021/04/16/chicag...,"<div class=""featured-video""><div class=""anvato...",Families Fight To Keep Memorial Trees Offered ...,**CHICAGO (CBS) -- **They have old photos and ...,t,2021-04-17 06:00:07.924868,2021-04-17 06:00:07.924881,100,CBS Chicago
422573,880832,NaN,https://wgntv.com/news/chicagocrime/18-year-ol...,"<div class=""article-content rich-text "">\n <p>...",18-year-old man in critical condition among 2 ...,CHICAGO _ — _ An 18-year-old man is in critica...,t,2021-04-17 04:01:25.630690,2021-04-17 04:01:25.630705,115,WGN Web Desk


TRAINED CODING

Rename columns so column names are unique after merge

In [13]:
trainedcoding_df.head()

,id,date,model_info,relevance,article_id,sentiment,bin,sentiment_processed
0,1616804,2020-03-24 03:01:07.336104+00,tagnews 1.2.4,0.999997,30864,NaN,0.0,t
1,1616839,2020-03-25 03:00:32.490177+00,tagnews 1.2.4,0.999879,30899,NaN,0.0,t
2,1616842,2020-08-26 03:01:08.9806+00,tagnews 1.2.4,0.998977,30902,NaN,1.0,t
3,1616854,2020-08-26 03:01:09.418435+00,tagnews 1.2.4,0.996706,30914,NaN,1.0,t
4,1616758,2020-02-06 23:42:49.97428+00,tagnews 1.2.4,0.199123,30818,NaN,9.0,f


In [14]:
trainedcoding_df.columns = ['id', 'date', 'model_info', 'overall_relevance',
                            'article_id', 'sentiment', 'bin', 'sentiment_processed']

In [15]:
trainedcategoryrelevance_df.head()

,id,relevance,category_id,coding_id
0,4271682,0.997400,23,1703704
1,4271683,0.992532,26,1703704
2,4271684,0.969676,2,1703704
3,4271685,0.940466,17,1703704
4,4271686,0.317523,2,1703705


In [16]:
trainedcategoryrelevance_df.columns = ['id','cat_relevance','category_id','coding_id']

Replace category IDs with category titles

In [17]:
categories_df.head()

,id,title,abbreviation,created,active,kind
0,15,Juvenile,JUVE,2011-08-26 20:22:06.828537+00,t,other
1,16,Re-Entry,REEN,2011-08-26 20:22:20.557875+00,t,other
2,17,Violence,VIOL,2011-08-26 20:22:33.659219+00,t,other
3,19,Probation,PROB,2011-08-26 20:23:14.245505+00,t,other
4,20,Parole,PARL,2011-08-26 20:23:20.964542+00,t,other


In [18]:
for row in categories_df.itertuples():
    trainedcategoryrelevance_df.loc[trainedcategoryrelevance_df['category_id'] == row.id,
                                    'category_id'] = row.title

In [19]:
trainedcategoryrelevance_df.head()

,id,cat_relevance,category_id,coding_id
0,4271682,0.997400,Chicago Public Schools,1703704
1,4271683,0.992532,Homicides,1703704
2,4271684,0.969676,Chicago Police Department,1703704
3,4271685,0.940466,Violence,1703704
4,4271686,0.317523,Chicago Police Department,1703705


Get trained codings with relevant crime categories

In [20]:
# Can drop sentiment related columns from trained coding; can drop 'id' column from categories
rel_trained_df = pd.merge(left=trainedcoding_df.drop(columns=['sentiment','bin','sentiment_processed']),
                         right=trainedcategoryrelevance_df.drop(columns=['id']),
                         left_on='id',
                         right_on='coding_id')
rel_trained_df = rel_trained_df[(rel_trained_df['cat_relevance'] >= CATEGORY_RELEVANCE_MIN)]

display(rel_trained_df.sort_values(by='article_id').head(10))

,id,date,model_info,overall_relevance,article_id,cat_relevance,category_id,coding_id
1294405,1585986,2020-02-06 23:31:57.851733+00,tagnews 1.2.4,0.758854,1,0.758854,Gun Violence,1585986
1294409,1585988,2020-02-06 23:31:57.904407+00,tagnews 1.2.4,0.796053,3,0.796053,Chicago Police Department,1585988
1216552,1585991,2020-02-06 23:31:57.957061+00,tagnews 1.2.4,0.978682,6,0.978682,Unspecified Crime,1585991
1216556,1585994,2020-02-06 23:31:58.015216+00,tagnews 1.2.4,0.866723,9,0.716029,Homicides,1585994
1216555,1585994,2020-02-06 23:31:58.015216+00,tagnews 1.2.4,0.866723,9,0.866723,Unspecified Crime,1585994
1216562,1585997,2020-02-06 23:31:58.069588+00,tagnews 1.2.4,0.535168,12,0.535168,Chicago Police Department,1585997
1294657,1585999,2020-11-27 03:00:59.448363+00,tagnews 1.2.4,0.934560,14,0.934560,Chicago Police Department,1585999
1216535,1586000,2020-11-27 03:00:59.967033+00,tagnews 1.2.4,0.963884,15,0.963884,Cook County Jail,1586000
1216536,1586000,2020-11-27 03:00:59.967033+00,tagnews 1.2.4,0.963884,15,0.896525,Chicago Police Department,1586000
1216537,1586000,2020-11-27 03:00:59.967033+00,tagnews 1.2.4,0.963884,15,0.837398,Gun Violence,1586000


Reshape long format of categories into wide

In [21]:
rel_trained_df['category_id'] = rel_trained_df['category_id'].astype('str')
rel_trained_df = pd.pivot_table(rel_trained_df,
                                index=['overall_relevance','article_id','coding_id'],
                                values='category_id',
                                aggfunc=lambda x: ', '.join(x)).reset_index()
display(rel_trained_df.sort_values(by='article_id').head(10))

,overall_relevance,article_id,coding_id,category_id
79799,0.758854,1,1585986,Gun Violence
92825,0.796053,3,1585988,Chicago Police Department
212585,0.978682,6,1585991,Unspecified Crime
121771,0.866723,9,1585994,"Unspecified Crime, Homicides"
11009,0.535168,12,1585997,Chicago Police Department
161672,0.934560,14,1585999,Chicago Police Department
189898,0.963884,15,1586000,"Cook County Jail, Chicago Police Department, G..."
93861,0.798977,16,1586001,"Chicago Police Department, Burlgary"
216695,0.980782,17,1586002,Unspecified Crime
347556,0.999992,18,1586003,"Chicago Police Department, Beat Realignment / ..."


In [22]:
# Indicate what minimum relevance cutoff was used for categories
rel_trained_df['cat_relevance_min'] = CATEGORY_RELEVANCE_MIN

display(rel_trained_df.sort_values(by='article_id').head(10))

,overall_relevance,article_id,coding_id,category_id,cat_relevance_min
79799,0.758854,1,1585986,Gun Violence,0.5
92825,0.796053,3,1585988,Chicago Police Department,0.5
212585,0.978682,6,1585991,Unspecified Crime,0.5
121771,0.866723,9,1585994,"Unspecified Crime, Homicides",0.5
11009,0.535168,12,1585997,Chicago Police Department,0.5
161672,0.934560,14,1585999,Chicago Police Department,0.5
189898,0.963884,15,1586000,"Cook County Jail, Chicago Police Department, G...",0.5
93861,0.798977,16,1586001,"Chicago Police Department, Burlgary",0.5
216695,0.980782,17,1586002,Unspecified Crime,0.5
347556,0.999992,18,1586003,"Chicago Police Department, Beat Realignment / ...",0.5


Get trained location data

In [23]:
# We only want the locations indicated as is_best
best_trained_loc_df = trainedlocation_df[trainedlocation_df['is_best'] == 't']

rel_trained_loc_df = pd.merge(how='left',
                              left=rel_trained_df,
                              right=best_trained_loc_df.drop(columns=['id','neighborhood']),
                              left_on='coding_id',
                              right_on='coding_id')

display(rel_trained_loc_df.sort_values(by='article_id').head(10))

,overall_relevance,article_id,coding_id,category_id,cat_relevance_min,text,latitude,longitude,confidence,is_best
80061,0.758854,1,1585986,Gun Violence,0.5,NaN,NaN,NaN,NaN,NaN
93144,0.796053,3,1585988,Chicago Police Department,0.5,NaN,NaN,NaN,NaN,NaN
213781,0.978682,6,1585991,Unspecified Crime,0.5,NaN,NaN,NaN,NaN,NaN
122195,0.866723,9,1585994,"Unspecified Crime, Homicides",0.5,NaN,NaN,NaN,NaN,NaN
11031,0.535168,12,1585997,Chicago Police Department,0.5,NaN,NaN,NaN,NaN,NaN
162386,0.934560,14,1585999,Chicago Police Department,0.5,NaN,NaN,NaN,NaN,NaN
190749,0.963884,15,1586000,"Cook County Jail, Chicago Police Department, G...",0.5,NaN,NaN,NaN,NaN,NaN
94182,0.798977,16,1586001,"Chicago Police Department, Burlgary",0.5,Damen Avenue exit,41.595223,-87.665472,0.74,t
217910,0.980782,17,1586002,Unspecified Crime,0.5,NaN,NaN,NaN,NaN,NaN
350287,0.999992,18,1586003,"Chicago Police Department, Beat Realignment / ...",0.5,NaN,NaN,NaN,NaN,NaN


Clean up dataframe

In [24]:
# Clean up columns
rel_trained_loc_df = rel_trained_loc_df.drop(['is_best','coding_id'], axis=1)
rel_trained_loc_df.rename(columns = {'confidence':'loc_confidence',
                                     'category_id':'categories',
                                     'text':'loc_text'},
                          inplace = True)
display(rel_trained_loc_df.sort_values(by='article_id').head(10))

,overall_relevance,article_id,categories,cat_relevance_min,loc_text,latitude,longitude,loc_confidence
80061,0.758854,1,Gun Violence,0.5,NaN,NaN,NaN,NaN
93144,0.796053,3,Chicago Police Department,0.5,NaN,NaN,NaN,NaN
213781,0.978682,6,Unspecified Crime,0.5,NaN,NaN,NaN,NaN
122195,0.866723,9,"Unspecified Crime, Homicides",0.5,NaN,NaN,NaN,NaN
11031,0.535168,12,Chicago Police Department,0.5,NaN,NaN,NaN,NaN
162386,0.934560,14,Chicago Police Department,0.5,NaN,NaN,NaN,NaN
190749,0.963884,15,"Cook County Jail, Chicago Police Department, G...",0.5,NaN,NaN,NaN,NaN
94182,0.798977,16,"Chicago Police Department, Burlgary",0.5,Damen Avenue exit,41.595223,-87.665472,0.74
217910,0.980782,17,Unspecified Crime,0.5,NaN,NaN,NaN,NaN
350287,0.999992,18,"Chicago Police Department, Beat Realignment / ...",0.5,NaN,NaN,NaN,NaN


PREP ARTICLES

Merge news source names into articles DF

In [25]:
articles_merge_df = pd.merge(left=articles_df,
                      right=newssource_df,
                      left_on='news_source_id',
                      right_on='source_id')

In [26]:
# Can drop columns from newsource and article DFs that are no longer needed
articles_merge_df = articles_merge_df.drop(['feedname',
                                            'source_id',
                                            'legacy_feed_id',
                                            'news_source_id',
                                            'relevant',
                                            'orig_html',
                                            'bodytext'], axis=1)

In [27]:
# Reorder DF so news source name is first after id
articles_merge_df = articles_merge_df[['id','source_name','short_name','url','title',
                                       'created','last_modified','author']]

MERGE ARTICLES WITH TRAINED CODING CATEGORIES AND LOCATION

In [28]:
articles_trained_merge_df = pd.merge(left=articles_merge_df,
                                     right=rel_trained_loc_df,
                                     left_on='id',
                                     right_on='article_id')
print(articles_trained_merge_df.shape)

(365890, 16)


In [29]:
articles_trained_merge_df.tail()

,id,source_name,short_name,url,title,created,last_modified,author,overall_relevance,article_id,categories,cat_relevance_min,loc_text,latitude,longitude,loc_confidence
365885,12507,Chicago News Cooperative,chicago-news-cooperative,http://www.chicagonewscoop.org/warren-loud-sch...,Warren: Loud Schools Debate Misses Mark,2012-02-28 00:16:27.258493,2013-09-03 17:07:07.625589,NaN,0.999498,12507,Chicago Public Schools,0.5,NaN,NaN,NaN,NaN
365886,12506,Chicago News Cooperative,chicago-news-cooperative,http://www.chicagonewscoop.org/councils-sarge-...,Council’s “Sarge” on Plan to Quiet Meetings,2012-02-28 00:16:26.217300,2012-11-10 21:02:35.764959,NaN,0.784052,12506,Criminal Justice Policy,0.5,City Hall. She,41.884346,-87.631668,0.882
365887,12505,Chicago News Cooperative,chicago-news-cooperative,http://www.chicagonewscoop.org/homeless-immigr...,"Homeless Immigrants Alone, Adrift",2012-02-28 00:16:25.188685,2013-09-03 17:10:01.018327,NaN,0.986434,12505,Immigration,0.5,NaN,NaN,NaN,NaN
365888,12504,Chicago News Cooperative,chicago-news-cooperative,http://www.chicagonewscoop.org/warren-posner-p...,Warren: Posner Ponders Yet Again,2012-02-28 00:16:24.149505,2012-11-10 21:02:35.431678,NaN,0.951153,12504,"Chicago Police Department, Unspecified Crime",0.5,NaN,NaN,NaN,NaN
365889,12502,Chicago News Cooperative,chicago-news-cooperative,http://www.chicagonewscoop.org/leo-track-runni...,Leo Track Running Despite Hurdles,2012-02-28 00:16:22.102951,2012-11-10 21:02:35.455707,NaN,0.548729,12502,Chicago Public Schools,0.5,NaN,NaN,NaN,NaN


### Filtering the data frame : 2017 - Present

In [45]:
articles_df_2019_present = articles_trained_merge_df[articles_trained_merge_df['created']>datetime.datetime(2019,1,1)]

In [46]:
articles_df_2019_present.tail()

,id,source_name,short_name,url,title,created,last_modified,author,overall_relevance,article_id,categories,cat_relevance_min,loc_text,latitude,longitude,loc_confidence
362372,271645,DNAInfo Chicago,dnainfo-chicago,https://www.dnainfo.com/chicago/20170101/rosel...,"1 Woman Fatally Stabbed, 1 Injured In 'Domesti...",2017-01-02 00:50:03.463167,2017-01-16 01:03:34.332797,NaN,0.780517,271645,"Homicides, Violence, Gun Violence",0.5,"9300 Block of South Wentworth Avenue,",41.724179,-87.708252,0.528
362373,271645,DNAInfo Chicago,dnainfo-chicago,https://www.dnainfo.com/chicago/20170101/rosel...,"1 Woman Fatally Stabbed, 1 Injured In 'Domesti...",2017-01-02 00:50:03.463167,2017-01-16 01:03:34.332797,NaN,0.780517,271645,"Homicides, Violence, Gun Violence",0.5,"9300 Block of South Wentworth Avenue,",41.724179,-87.708252,0.528
362374,271592,DNAInfo Chicago,dnainfo-chicago,https://www.dnainfo.com/chicago/20170101/uptow...,Two Men Fatally Shot In Uptown Identified - U...,2017-01-01 18:50:05.508174,2017-01-15 19:03:07.789009,NaN,0.999842,271592,"Gun Violence, Homicides, Violence",0.5,4600 block of North Broadway. A,41.958140,-87.657071,0.867
362375,271591,DNAInfo Chicago,dnainfo-chicago,https://www.dnainfo.com/chicago/20170101/west-...,Accused Rapist Marc Winner Involved In Dispute...,2017-01-01 18:50:04.364244,2017-01-15 19:03:07.738909,NaN,0.981288,271591,"Chicago Police Department, Sexual Assault, Vio...",0.5,Loop.,41.877713,-87.626313,1.000
362376,271496,DNAInfo Chicago,dnainfo-chicago,https://www.dnainfo.com/chicago/20161231/aubur...,"Shootings Kill 3 People, Wound 8 Since Friday,...",2017-01-01 00:50:10.426879,2017-01-15 01:03:40.899802,NaN,0.999779,271496,"Gun Violence, Homicides, Robbery, Violence, Ga...",0.5,8200 block of South Ingleside Avenue,41.745697,-87.601743,1.000


### Refining the data-frame

In [47]:
articles_df_2019_present = articles_df_2019_present[['source_name','short_name',
                                                 'url','title','created','last_modified',
                                                 'categories','loc_text','latitude','longitude','loc_confidence']]

In [48]:
articles_df_2019_present.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 256306 entries, 0 to 362376
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   source_name     256306 non-null  object        
 1   short_name      256306 non-null  object        
 2   url             256306 non-null  object        
 3   title           256301 non-null  object        
 4   created         256306 non-null  datetime64[ns]
 5   last_modified   256306 non-null  datetime64[ns]
 6   categories      256306 non-null  object        
 7   loc_text        84681 non-null   object        
 8   latitude        84681 non-null   float64       
 9   longitude       84681 non-null   float64       
 10  loc_confidence  84681 non-null   float64       
dtypes: datetime64[ns](2), float64(3), object(6)
memory usage: 23.5+ MB


In [49]:
articles_df_2019_present.head()

,source_name,short_name,url,title,created,last_modified,categories,loc_text,latitude,longitude,loc_confidence
0,Crains,crains,http://www.chicagobusiness.com/health-care/all...,All adults in Chicago are eligible for a COVID...,2021-04-17 00:01:26.613838,2021-04-17 00:01:26.613852,Gun Violence,NaN,NaN,NaN,NaN
1,Crains,crains,http://www.chicagobusiness.com/opinion/time-ch...,Time for Chicago business leaders to start map...,2021-04-16 22:01:51.433588,2021-04-16 22:01:51.433603,Violence,NaN,NaN,NaN,NaN
2,Crains,crains,http://www.chicagobusiness.com/tourism/pent-tr...,"As pent-up travel demand explodes, Chicago's t...",2021-04-16 22:01:39.741106,2021-04-16 22:01:39.741124,Homicides,NaN,NaN,NaN,NaN
3,Crains,crains,http://www.chicagobusiness.com/government/kinz...,"Kinzinger, fellow House impeachers see donatio...",2021-04-16 20:01:27.640738,2021-04-16 20:01:27.640753,Fraud,NaN,NaN,NaN,NaN
4,Crains,crains,http://www.chicagobusiness.com/greg-hinz-polit...,We all failed Adam Toledo,2021-04-16 18:01:43.857139,2021-04-16 18:01:43.857152,Gun Violence,NaN,NaN,NaN,NaN


In [50]:
articles_df_2019_present.tail()

,source_name,short_name,url,title,created,last_modified,categories,loc_text,latitude,longitude,loc_confidence
362372,DNAInfo Chicago,dnainfo-chicago,https://www.dnainfo.com/chicago/20170101/rosel...,"1 Woman Fatally Stabbed, 1 Injured In 'Domesti...",2017-01-02 00:50:03.463167,2017-01-16 01:03:34.332797,"Homicides, Violence, Gun Violence","9300 Block of South Wentworth Avenue,",41.724179,-87.708252,0.528
362373,DNAInfo Chicago,dnainfo-chicago,https://www.dnainfo.com/chicago/20170101/rosel...,"1 Woman Fatally Stabbed, 1 Injured In 'Domesti...",2017-01-02 00:50:03.463167,2017-01-16 01:03:34.332797,"Homicides, Violence, Gun Violence","9300 Block of South Wentworth Avenue,",41.724179,-87.708252,0.528
362374,DNAInfo Chicago,dnainfo-chicago,https://www.dnainfo.com/chicago/20170101/uptow...,Two Men Fatally Shot In Uptown Identified - U...,2017-01-01 18:50:05.508174,2017-01-15 19:03:07.789009,"Gun Violence, Homicides, Violence",4600 block of North Broadway. A,41.958140,-87.657071,0.867
362375,DNAInfo Chicago,dnainfo-chicago,https://www.dnainfo.com/chicago/20170101/west-...,Accused Rapist Marc Winner Involved In Dispute...,2017-01-01 18:50:04.364244,2017-01-15 19:03:07.738909,"Chicago Police Department, Sexual Assault, Vio...",Loop.,41.877713,-87.626313,1.000
362376,DNAInfo Chicago,dnainfo-chicago,https://www.dnainfo.com/chicago/20161231/aubur...,"Shootings Kill 3 People, Wound 8 Since Friday,...",2017-01-01 00:50:10.426879,2017-01-15 01:03:40.899802,"Gun Violence, Homicides, Robbery, Violence, Ga...",8200 block of South Ingleside Avenue,41.745697,-87.601743,1.000


EXPORT TO LOCAL FILE SYSTEM (.CSV)

In [68]:
articles_df_2019_present.to_csv("histroical_data.csv",index=False)

In [56]:
articles_df_2019_present.info()

NameError: name 'articles_df_2017_present' is not defined

In [70]:
print("--- %s seconds ---" % (time.time() - start_time),' For the complete Process')
print("--- %s seconds ---" % (downloading_time_end - downloading_time_start),' For Downloading File from S3')
print("--- %s seconds ---" % (download_and_extracting_time_end - download_and_extracting_time_start),' For the Downlading and extraction File')
print("--- %s seconds ---" % (data_loading_time_end - data_loading_time_start),' For Loading the data into the notebook')

--- 399.99418473243713 seconds ---  For the complete Process
--- 157.40076684951782 seconds ---  For Downloading File from S3
--- 51.5209596157074 seconds ---  For the Downlading and extraction File
--- 170.27853631973267 seconds ---  For Loading the data into the notebook


### Importing Final Data to Google Sheets

In [51]:

import pygsheets
import numpy as np

In [86]:
def write_gsheet(df,wkb_name):
    
    gc = pygsheets.authorize(service_account_file='GCP_credentials_for_gheets/chicagojusticeprojectscraping-7891c0e9a0db.json')
    sh = gc.open(wkb_name)
    sheet_num = str(sh.worksheets()[0])[17]
    rows_df,columns_df = df.shape
    
    if rows_df*columns_df >= 50000000:
        return False

    elif sheet_num == '1':
        wks = sh.add_worksheet("Sheet3",rows=2 ,cols=2,index=0)
        wks = sh.worksheet_by_title("Sheet1")
        sh.del_worksheet(wks)
        
        wks = sh.add_worksheet("Sheet2",rows=rows_df ,cols=columns_df,index=0)
        wks = sh.worksheet_by_title("Sheet2")
        wks.set_dataframe(articles_df_2019_present,(1,1))
        
        wks = sh.worksheet_by_title("Sheet3")
        sh.del_worksheet(wks)
        return True
    
    else:
        wks = sh.add_worksheet("Sheet3",rows=2 ,cols=2,index=0)
        wks = sh.worksheet_by_title("Sheet2")
        sh.del_worksheet(wks)
        
        wks = sh.add_worksheet("Sheet1",rows=rows_df ,cols=columns_df,index=0)
        wks = sh.worksheet_by_title("Sheet1")
        wks.set_dataframe(articles_df_2019_present,(1,1))
        
        wks = sh.worksheet_by_title("Sheet3")
        sh.del_worksheet(wks)
        return True
    

In [87]:
%%time
write_gsheet(articles_df_2019_present,'Scraped Data')

Wall time: 4min 15s


True